In [21]:
# meant to be done on google colab. dataset should be in your google drive

In [22]:
!pip install torch
!pip install torchvision
!pip install pandas

In [23]:
!unzip /content/drive/MyDrive/plant_identifier_project/dataset.zip

Archive:  /content/drive/MyDrive/plant_identifier_project/dataset.zip
replace dataset_type_of_plants_new/aloevera/aloevera0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset_type_of_plants_new/aloevera/aloevera0.jpg  
replace dataset_type_of_plants_new/aloevera/aloevera1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset_type_of_plants_new/aloevera/aloevera1.jpg  
replace dataset_type_of_plants_new/aloevera/aloevera10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset_type_of_plants_new/aloevera/aloevera10.jpg  
replace dataset_type_of_plants_new/aloevera/aloevera100.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset_type_of_plants_new/aloevera/aloevera100.jpg  
replace dataset_type_of_plants_new/aloevera/aloevera101.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset_type_of_plants_new/aloevera/aloevera101.jpg  
replace dataset_type_of_plants_new/aloevera/aloevera102.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y

In [24]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import Subset
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import os

In [25]:
# Define the path to the main folder containing subfolders of each class
data_dir = '/content/dataset_type_of_plants_new'

# Define the transformation to apply to the input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the input image to 224x224
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Create an instance of the ImageFolder dataset class
dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)
image, label = dataset[0]
print(image.shape)

torch.Size([3, 224, 224])


In [26]:
# Split the dataset into training, validation, and testing sets
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate the number of samples for each split
num_samples = len(dataset)
num_train = int(train_ratio * num_samples)
num_val = int(val_ratio * num_samples)
num_test = num_samples - num_train - num_val

# Use random_split to split the dataset into train, val, and test subsets
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [num_train, num_val, num_test])

In [27]:
# Create data loaders for training, validation, and testing sets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [28]:
# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

In [29]:
# Modify the last fully connected layer to match the number of classes in your plant dataset
num_ftrs = model.fc.in_features
num_classes = len(dataset.classes)
model.fc = nn.Linear(num_ftrs, num_classes)

In [30]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [31]:
# Move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [32]:
# Train the model
num_epochs = 40
best_val_acc = 0.0  # Initialize the best validation accuracy
best_model_path = 'best_model.pt'  # Path to save the best model

In [33]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0

    # Use tqdm to wrap the train_loader for progress bar visualization
    with tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as t:
        for images, labels in t:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            train_correct += torch.sum(preds == labels.data)

            # Update tqdm progress bar description
            t.set_postfix({'train_loss': train_loss / ((t.n + 1) * images.size(0))})

    train_loss = train_loss / len(train_dataset)
    train_acc = train_correct.double() / len(train_dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            val_correct += torch.sum(preds == labels.data)

    val_loss = val_loss / len(val_dataset)
    val_acc = val_correct.double() / len(val_dataset)

    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}'.format(
        epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc))

    # Check if the current model has a better validation accuracy than the previous best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model, best_model_path)
        print('Best model saved with validation accuracy: {:.4f}'.format(best_val_acc))


Epoch 1/40: 100%|██████████| 750/750 [04:45<00:00,  2.63batch/s, train_loss=2.76]


Epoch [1/40], Train Loss: 2.7552, Train Acc: 0.1966, Val Loss: 2.3301, Val Acc: 0.2900
Best model saved with validation accuracy: 0.2900


Epoch 2/40: 100%|██████████| 750/750 [04:35<00:00,  2.73batch/s, train_loss=2.11]


Epoch [2/40], Train Loss: 2.1052, Train Acc: 0.3594, Val Loss: 1.9457, Val Acc: 0.4240
Best model saved with validation accuracy: 0.4240


Epoch 3/40: 100%|██████████| 750/750 [04:35<00:00,  2.72batch/s, train_loss=1.74]


...

Epoch [12/40], Train Loss: 0.2198, Train Acc: 0.9149, Val Loss: 0.6510, Val Acc: 0.8343
Best model saved with validation accuracy: 0.8343


Epoch 13/40: 100%|██████████| 750/750 [04:34<00:00,  2.73batch/s, train_loss=0.19]


Epoch [13/40], Train Loss: 0.1902, Train Acc: 0.9222, Val Loss: 0.8193, Val Acc: 0.8060


Epoch 14/40:  89%|████████▉ | 666/750 [04:01<00:30,  2.75batch/s, train_loss=0.19]


KeyboardInterrupt: ignored

In [34]:
# Load the saved best model
best_model_path = 'best_model.pt'
model = torch.load(best_model_path)
model.eval()

test_correct = 0

# Testing
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        test_correct += torch.sum(preds == labels.data)

test_acc = test_correct.double() / len(test_dataset)

print('Test Accuracy: {:.4f}'.format(test_acc))

Test Accuracy: 0.8280
